In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import os
import pandas as pd

from src.evaluation.metrics import aggregate_scores, calculate_scores
from src.data.data_loading import load_config
from src.data.code_processor import parse_code

In [3]:
# set working directory to project root - EXECUTE ONLY ONCE or RESTART KERNEL
os.chdir('..')
os.getcwd()

'C:\\Users\\merti\\PycharmProjects\\cadenza-playwright-llm'

In [4]:
config = load_config(config_path='config/config.yaml')

# Scoring
Metrics implemented and used in this project (see `src/evaluation/metrics.py`):
* **Weighted BLEU** $ \in [0.0, 1.0] $: The BLEU score proposed by [Papineni et al. (2002)](https://aclanthology.org/P02-1040.pdf) [1], [2] is a metric that measures the similarity between two sequences of text. The weighted BLEU score is a variant implementd in this project that uses a weighted average of the BLEU scores of the precondition part and the actual generated additional part in teh generated test script. The weights are defined in the configuration file `config/config.yaml`.
* **Success Rate** $ \in [0.0, 1.0] $: The success rate is the proportion of generated test scripts that run successfully, no matter if they are semantically correct or not.
* **Levenshtein Distance** $ d(s, t) \in \mathbb{N} $: The Levenshtein distance between strings $ s $ and $ t $ is an integer that measures the minimum number of single-character edits (insertions, deletions, or substitutions) required to change $ s $ into $ t $.

In [5]:
# Define example test cases
test_cases = [
    {
        'generated_code': parse_code(config['paths']['prediction_dir']+'/1_2.pred.ts'),
        'validation_code': parse_code(config['dataloading']['test_script_dir']+'/1_2.spec.ts'),
        'precondition_code': parse_code(config['dataloading']['test_script_dir']+'/1_1.spec.ts')
    },
    {
        'generated_code': parse_code(config['paths']['prediction_dir']+'/2_2.pred.ts'),
        'validation_code': parse_code(config['dataloading']['test_script_dir']+'/2_2.spec.ts'),
        'precondition_code': parse_code(config['dataloading']['test_script_dir']+'/2_1.spec.ts')
    }
]

2024-07-14 11:27:22 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 16
2024-07-14 11:27:22 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 31
2024-07-14 11:27:22 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 32
2024-07-14 11:27:22 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 12
2024-07-14 11:27:22 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 13
2024-07-14 11:27:22 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 13


In [6]:
# Run for one test case
scores = calculate_scores([test_cases[0]])
scores

{'weighted bleu': [0.4912019643332622],
 'success rate': [None],
 'levenshtein distance': [0.33620689655172414]}

In [7]:
# Run for all test cases
scores = calculate_scores(test_cases)
scores

{'weighted bleu': [0.4912019643332622, 0.3995064945165301],
 'success rate': [None, None],
 'levenshtein distance': [0.33620689655172414, 0.2632398753894081]}

In [8]:
# Aggregate scores
agg_scores = aggregate_scores(scores)
agg_scores

2024-07-14 11:27:22 [src.evaluation.metrics:31] [WARNING] >>>> Missing success rate scores. Skipping...


{'weighted bleu': 0.4453542294248961,
 'success rate': None,
 'levenshtein distance': 0.29972338597056614}

In [9]:
# Run complete evaluation script
!python scripts/evaluation.py --config=config/config.yaml

2024-07-14 11:27:24 [__main__:21] [INFO] >>>> Calculating scores...
2024-07-14 11:27:24 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 16
2024-07-14 11:27:24 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 31
2024-07-14 11:27:24 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 32
2024-07-14 11:27:24 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 12
2024-07-14 11:27:24 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 13
2024-07-14 11:27:24 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 13
2024-07-14 11:27:24 [__main__:40] [INFO] >>>> Scores calculated. Aggregating...
2024-07-14 11:27:24 [src.evaluation.metrics:31] [WARNING] >>>> Missing success rate scores. Skipping...
2024-07-14 11:27:24 [__main__:42] [INFO] >>>> Aggregated scores: {'weighted bleu': 0.445354229424896

In [10]:
# Load evaluation results
results = pd.read_pickle(config['paths']['scores_dir']+'eval_scores_20240714-1123.pkl')
results

,file_id,test_case,test_step,weighted bleu,success rate,levenshtein distance
0,1_2,1,2,0.445354,None,0.299723
1,2_2,2,2,0.445354,None,0.299723


# References
[1]
```bibtex
@INPROCEEDINGS{Papineni02bleu:a,
    author = {Kishore Papineni and Salim Roukos and Todd Ward and Wei-jing Zhu},
    title = {BLEU: a Method for Automatic Evaluation of Machine Translation},
    booktitle = {},
    year = {2002},
    pages = {311--318}
}
```
[2]
```bibtex
@inproceedings{lin-och-2004-orange,
    title = "{ORANGE}: a Method for Evaluating Automatic Evaluation Metrics for Machine Translation",
    author = "Lin, Chin-Yew  and
      Och, Franz Josef",
    booktitle = "{COLING} 2004: Proceedings of the 20th International Conference on Computational Linguistics",
    month = "aug 23{--}aug 27",
    year = "2004",
    address = "Geneva, Switzerland",
    publisher = "COLING",
    url = "https://www.aclweb.org/anthology/C04-1072",
    pages = "501--507",
}
```
